In [1]:
max1demand=11    #use 1 more than actual demand
max2demand=11    #use 1 more than actual demand
sparti=[]  #start with null #ZoneActiveStateDefinition 1-Not active in 1; 2-Not active in 2; 3- Active in 1; 4- Active in 2;
spartj=[]  #start with null #Zone1DemandStateDef
spartk=[]  #start with null #Zone2DemandStateDef
spartl=[]  #start with null #PreviousZoneStateDef
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:4           #Create an index to call   
    for j in 1:max1demand
        for k in 1:max2demand
            for l in 1:2 #2 regions
                sparti= push!(sparti,i)  #for i in ijk add to sparti definition
                spartj= push!(spartj,j)  #for j in ijk add to spartj definition
                spartk= push!(spartk,k)  #for k in ijk add to spartk definition
                spartl= push!(spartl,l)  #for k in ijk add to spartk definition
                sidx= push!(sidx,s)      #create index in sidx
                s=s+1
            end
        end
    end
end

#sparti
#spartj
#spartk
#[sparti,spartj,spartk]
#sidx

#idxactive=  (sparti.>=3) .& (spartj+spartk.==2) ##(Wrapped into regional indices)
idxidle= (sparti.<3) .& (spartj+spartk .>2)   #index all ijk where idling cannot occur
idxZoneFrom= (sparti.==1) .& (spartl.==2) .| (sparti.==2) .& (spartl.==1) #idx where zone from cannot occur
idxregion1= (sparti.==3) .& (spartj.==1)      #index all ijk where region 1 is active but there are no calls in region 1 to service
idxregion2= (sparti.==4) .& (spartk.==1)      #index all ijk where region 2 is active but there are no calls in region 2 to service


idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxZoneFrom   #index of the previous 3 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
sidx= [1:length(sparti);];          #number of states
sidx

442-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 431
 432
 433
 434
 435
 436
 437
 438
 439
 440
 441
 442

In [2]:
λ=1.75
μ1=5
μ2=4
μMatr=[μ1 μ2; 
       μ2 μ1]

2×2 Matrix{Int64}:
 5  4
 4  5

In [3]:
N=["λ" "μ1" "μ2" "Throughput" "Deliveries/Hour" "Avg. # in Sys" "Avg. # Variance" "Avg. Time in Sys" "% of Time 0 calls" "% of Time 1 call" "Compliance Rate"]
μ1=5.5
λ=0 #Define overall Arrival Rate for every zone


for i in 1:1
     λ= λ+.5
     μ1=5.5
  for j in 1:8
      μ1=μ1-.5

μ2=.9*μ1
μMatr=[μ1 μ2; 
       μ2 μ1]
matrix= zeros(length(sidx),length(sidx))
λ1idx=zeros(length(sidx))
λ2idx=zeros(length(sidx)) 
μMat= zeros(length(sidx),1)
μidx=[]
λ1=λ  #Define Arrival Rate for Zone1
λ2=λ1  #Define Arrival Rate for Zone2                

for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    RegionFrom= spartl[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nexts=0
    RegionService= 0    
    nextRegionFrom=0

#  Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        if Zoneflag <=2
            nextZoneflag = 3 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end

        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
         λ1idx[s]=1
    end
    #Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1       
      if Zoneflag <=2
            nextZoneflag = 4 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ2
        matrix[s,s]= matrix[s,s] - λ2
        λ2idx[s]=1
    end
    
         #Service from Region to Region
    if Zoneflag == 3
        if Call1-1 >=max(Call2)/2 && Call1-1 >1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=3
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)        
        elseif Call1-1 >max(Call2)/2 && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)            
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag= 4
            nextCall1= Call1-1
            nextCall2= Call2
            nextRegionFrom=1
            μidx=push!(μidx,2)
        end    
    elseif Zoneflag == 4
        if Call2-1 >=max(Call1)/2 && Call2-1 >1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=4
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)           
        elseif Call2-1 >max(Call1)/2 && Call2-1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=2
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag= 3
            nextCall1= Call1
            nextCall2= Call2-1  
            nextRegionFrom=2
            μidx=push!(μidx,2)
        end
    else 
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextRegionFrom=RegionFrom
        μidx=push!(μidx,0)
    end    
  idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .&(spartl.==nextRegionFrom)
  nexts= sidx[idx] 
  matrix[nexts,s]=matrix[nexts,s].+RegionService
  matrix[s,s]= matrix[s,s].- RegionService   
    μMat[s,1]= μMat[s,1] .+ RegionService
 end
matrix[length(sidx),:].= 1 #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10) #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
M=hcat(sidx,A,B,C,D,X)     #create readable matrix for state, sparti,spartj,spartk, and pi
Throughput=(λ)*sum(M[λ1idx.==1,6])+(λ)*sum(M[λ2idx.==1,6])
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]-2)*X[s]
end
LVar=0
for s in 1:length(sidx)
  LVar=LVar+(B[s]+C[s]-2)^2*X[s]
end
LVar=LVar-L^2        
W=L/Throughput 
#Percentage of time with zero calls
Midx0= (spartj .+ spartk).==2
ZeroCall=sum(M[Midx0.==1,6])
#Percentage of time with one calls
Midx1= (spartj .+ spartk).==3
OneCall=sum(M[Midx1.==1,6])
CR= ZeroCall+OneCall            
N=[N; λ μ1 μ2 Throughput Gain L LVar W ZeroCall OneCall CR]
    end
end            

In [4]:
N

9×11 Matrix{Any}:
  "λ"   "μ1"   "μ2"   "Throughput"   "Deliveries/Hour"  …   "Compliance Rate"
 0.5   5.0    4.5    1.0            1.0                    0.955666
 0.5   4.5    4.05   1.0            1.0                    0.945326
 0.5   4.0    3.6    1.0            1.0                    0.930903
 0.5   3.5    3.15   1.0            1.0                    0.909926
 0.5   3.0    2.7    1.0            1.0                 …  0.877741
 0.5   2.5    2.25   0.999999       0.999999               0.824705
 0.5   2.0    1.8    0.999984       0.999984               0.728133
 0.5   1.5    1.35   0.999312       0.999312               0.524727

In [5]:
using DataFrames
df= DataFrame(N[2:Int(length(N[:,1])),:], :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.5,5.0,4.5,1.0,1.0,0.266904,0.337421,0.266904,0.789159,0.166507
2,0.5,4.5,4.05,1.0,1.0,0.305395,0.397513,0.305395,0.765806,0.179521
3,0.5,4.0,3.6,1.0,1.0,0.356812,0.482198,0.356812,0.736648,0.194255
4,0.5,3.5,3.15,1.0,1.0,0.428951,0.609471,0.428951,0.699224,0.210702
5,0.5,3.0,2.7,1.0,1.0,0.53743,0.819306,0.53743,0.649455,0.228286
6,0.5,2.5,2.25,0.999999,0.999999,0.718748,1.21902,0.718749,0.580082,0.244624
7,0.5,2.0,1.8,0.999984,0.999984,1.08239,2.20161,1.08241,0.476859,0.251274
8,0.5,1.5,1.35,0.999312,0.999312,2.15734,6.29328,2.15882,0.308276,0.216451


In [6]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\MD1AmbStats29MAR_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\MD1AmbStats29MAR_1.csv"